In [102]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [103]:
offers = pd.read_excel("data.xlsx",sheetname=0)  # Read Offers data
offers.head()

,Offer #,Campaign,Varietal,Minimum Qty (kg),Discount (%),Origin,Past Peak
0,1,January,Malbec,72,56,France,False
1,2,January,Pinot Noir,72,17,France,False
2,3,February,Espumante,144,32,Oregon,True
3,4,February,Champagne,72,48,France,True
4,5,February,Cabernet Sauvignon,144,44,New Zealand,True


In [104]:
offers.shape

(32, 7)

In [105]:
offers.columns = ["offer_id","campaign","varietal","min_qty","discount","origin","past_peak"] # Rename column names to be simple
offers.head()

,offer_id,campaign,varietal,min_qty,discount,origin,past_peak
0,1,January,Malbec,72,56,France,False
1,2,January,Pinot Noir,72,17,France,False
2,3,February,Espumante,144,32,Oregon,True
3,4,February,Champagne,72,48,France,True
4,5,February,Cabernet Sauvignon,144,44,New Zealand,True


In [106]:
transactions = pd.read_excel("data.xlsx",sheetname=1)  # Read Transactional data
transactions.head()

,Customer Last Name,Offer #
0,Smith,2
1,Smith,24
2,Johnson,17
3,Johnson,24
4,Johnson,26


In [107]:
transactions.columns = ["customer_name", "offer_id" ] # Rename columns
transactions["temp"]=1  # Add an extra column and populate with 1 so that it can be used later for counting 
transactions.head()

,customer_name,offer_id,temp
0,Smith,2,1
1,Smith,24,1
2,Johnson,17,1
3,Johnson,24,1
4,Johnson,26,1


### Merge Offers data with Transaction data

In [108]:
data = pd.merge(offers, transactions)  # Join the data from two data frames
data.head()

,offer_id,campaign,varietal,min_qty,discount,origin,past_peak,customer_name,temp
0,1,January,Malbec,72,56,France,False,Thomas,1
1,1,January,Malbec,72,56,France,False,Jackson,1
2,1,January,Malbec,72,56,France,False,Mitchell,1
3,1,January,Malbec,72,56,France,False,Peterson,1
4,1,January,Malbec,72,56,France,False,Wood,1


### Create Pivot table based on the Customer Name and Offers accepted

In [109]:
pivot = data.pivot_table(index=['customer_name'], columns=["offer_id"], values="temp") 
pivot.head()

offer_id,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
customer_name,,,,,,,,,,,,,,,,,,,,,
Adams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
Allen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
Anderson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
Bailey,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
Baker,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [110]:
pivot = pivot.fillna(0) # Replace "NaN" with 0
pivot.head()

offer_id,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
customer_name,,,,,,,,,,,,,,,,,,,,,
Adams,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
Allen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Anderson,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Bailey,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Baker,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [111]:
pivot = pivot.reset_index() # Convert "customer_name" index into a column of the data frame and add a numbered index
save_cols = pivot.columns[1:]  # Save columns list for future use
pivot.head()

offer_id,customer_name,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,Adams,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,Allen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Anderson,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bailey,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,Baker,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Build K-Means Model 

In [112]:
cluster = KMeans(n_clusters=5)
pivot['cluster'] = cluster.fit_predict(pivot[pivot.columns[2:]]) # Pass columns related to offer ids only for clustering
pivot.cluster.value_counts()

4    34
2    24
0    19
1    12
3    11
Name: cluster, dtype: int64

### Use PCA to transform offer id columns into 2D data set

In [113]:
pca = PCA(n_components=2)
pivot["x"] = pca.fit_transform(pivot[save_cols])[:,0]
pivot["y"] = pca.fit_transform(pivot[save_cols])[:,1]
pivot = pivot.reset_index()

In [114]:
customer_segments = pivot[["customer_name","cluster","x","y"]]
customer_segments.head()

offer_id,customer_name,cluster,x,y
0,Adams,2,1.007580,0.108215
1,Allen,4,-0.287539,0.044715
2,Anderson,1,-0.392032,1.038391
3,Bailey,2,0.699477,-0.022542
4,Baker,4,0.088183,-0.471695


In [117]:
data_set = pd.merge(transactions, customer_segments)
data_set = pd.merge(offers, df)

In [130]:
data_set["is_2"] = data_set.cluster==2
data_set.groupby("is_2").varietal.value_counts() # Shows the varietal counts for cluster 2 vs all other clusters

is_2   varietal          
False  Champagne             81
       Pinot Noir            44
       Cabernet Sauvignon    32
       Prosecco              31
       Espumante             18
       Malbec                16
       Chardonnay            15
       Merlot                14
       Pinot Grigio           2
True   Espumante             22
       Malbec                16
       Pinot Grigio          15
       Prosecco              12
       Merlot                 6
Name: varietal, dtype: int64

### From the above break down, we can see almost all of the "Pinot Grigio" offers were purchased by Cluster 2. Also, none of the "Champagne", "Pinot Noir" and "Cabernet Sauvignon" offers were purchased by Cluster 2.

In [135]:
data_set.groupby("is_2")[['min_qty', 'discount']].mean()

,min_qty,discount
is_2,,
False,73.114625,60.553360
True,6.000000,55.661972


### Based on the average of minimum quantity purchased, it looks like Customers of cluster 2 make purchases in low quantities.